In [12]:
#2019.12.3  目标：将当前时间的分钟数据与历史分钟数据进行对比，预测股票趋势。本程序名字为:
from sqlalchemy import create_engine
import pymysql
from datetime import datetime,timedelta
import tushare as ts
import pandas as pd 
pro=ts.pro_api()

#给出回溯天数，以当前日期向前回溯，算出交易开始时间，以列表形式返回开始到现在的所有交易日期，日期格式为20191120.

def tradedays_stock(ndays):
#tushare也有tradedays相近的函数名，加一个stock，防止函数名冲突
    day=datetime.today()
    #取今天的时间
    end_date=day.strftime('%Y%m%d')
    #默认输出是2019-10-24的格式，用strftime换算成20191024
    start_date=(day-timedelta(days=ndays)).strftime('%Y%m%d')
    #开始时间根据ndays的不同，可以变化
    cal_date=pro.trade_cal(exchange='', start_date=start_date, end_date=end_date)
    #获取各大交易所交易日历数据,默认提取的是上交所
    trade_days=cal_date[cal_date.is_open==1].cal_date
    #is_open=1，表明该日期开市交易.trade_date为为指定时间内的交易日列表
    return list(trade_days)
    #返回类型是pandas.core.series.Series，可以用list转化为列表


#两个参数：股市交易日期列表，股票代码。读出指定代码，指定日期下的mysql数据。
def stock_min_data(trade_dates,stockcode):
    datalist=[]
    for dates in trade_dates:
        minnow=str(int(datetime.today().strftime('%H%M'))-30)
        #取当前时间前一分钟的小时与分钟数
        tablename='min'+dates+minnow
        dbname='stockmin'+dates[0:6]
        #print(dbname)
        db = pymysql.connect("106.12.3.198","root","prophet2lmh",dbname)
        #建立数据库连接
        cursor = db.cursor()
        #建立游标

        #pymysql读出的数据没有包含列名，因此要先读出列名，sql中的命令为"show columns from tablename"
        sqlcommand='show columns from '+tablename+';'
        #取出列名
        cursor.execute(sqlcommand)
        data = cursor.fetchall()
        column_name=[]
        for i in data:
            column_name.append(i[0])
        #查询表格的列名并保存到列表中
        
        
        #下面的程序读出数据内容
        sqlcommand='select * from '+tablename+' where 代码='+stockcode+';'
        cursor.execute(sqlcommand)       
        data=cursor.fetchone()
        # 使用 fetchone() 方法获取单条数据.
        datalist.append(list(data)[1:])
        #将每次的数据增加到datalist中
        
        cursor.close()
        #关闭游标
        db.close()
        # 关闭数据库连接
        
    df=pd.DataFrame(datalist,columns=column_name[1:])
    return df


stock_min_data(tradedays_stock(1),'000001')


ProgrammingError: (1146, "Table 'stockmin201912.min201912021099' doesn't exist")

In [8]:

minnow


'1124'